In [ ]:
import os
import json
from openai import OpenAI
from pydantic import BaseModel, Field
from dotenv import load_dotenv

In [ ]:
class AssessmentResultPerCriteria(BaseModel):
    explanation: str = Field(..., description="A detailed reasoning that supports the decision, based on evidence from the document.")
    result: str = Field(..., description="The overall decision for this item. Respond only with one of ['yes', 'no'].")

In [ ]:
key = ""
client = OpenAI(api_key=key)

## Prompts

In [ ]:
intro_prompt = """
# Introduction and Role Setting:
You are a systematic reviewer who is an expert in risk of bias assessment of papers in environmental policies. You are particularly good at learning evaluation criteria, and closely following it to assess the risk of bias of climate and energy studies. 
You can fully understand and follow the evaluation guidelines and evaluate the studies I have provided to you. Make sure all your judgments are based on the facts reported in the article and not on any extrapolation or speculation of your own.
## Guidelines for Evaluation:
**Note:** The examples provided do not cover all possible scenarios in real-world applications. So, use your expert judgment to evaluate each item based on the information provided in the study, and do not rely solely on the examples.
### Important:
- If there is too little information to support the judgment, do not speculate positively.
- Reply with **ONLY** one of **Yes** or **No**. You provide reasoning behind each decision.
"""

In [ ]:
criteria_prompt = """
# Risk of Bias Assessment Criteria

### 2.3) Sample Exclusion
After the start of the intervention/exposure or during the analysis, were any subjects or areas excluded or lost from the study or analysis? When some subjects or areas, or collected data are excluded, it might increase the risk of post-intervention/exposure selection bias.
Consider the following:
- If the final participated sample is lower than the total potential participation sample, then respond with **yes**. 
- If the final participated sample is the same as the total potential participation sample, then respond with **no**.
"""

In [ ]:
output_format = """

# Output Format

Please output the result STRICTLY in the format below:

{
  "explanation": str,
  "result": str
}

where explanation is a detailed reasoning that supports the decision, based on evidence from the document, and result is the overall decision for this item, respond only with one of ['yes', 'no'].
"""

intro_and_output = "\n".join([intro_prompt, output_format])

# Assessment

In [ ]:
folder = "files"
file_name = "Houde, S; Todd, A; Sudarshan, A; Flora, JA; Armel, KC (2013).md"

with open(os.path.join(folder, file_name), "r", encoding="utf-8") as f:
    document = f.read()

In [ ]:
response = client.responses.create(
    model='gpt-4o',
    temperature=0,
    instructions=intro_prompt,
    input=[
        {
            "role": "user",
            "content": [
                {
                    "type": "input_text",
                    "text": f"{criteria_prompt}\n"
                },
                {
                    "type": "input_text",
                    "text": f"\nHere is the paper:\n{document}"
                }
            ]
        },
    ]
)

response_text = response.output_text

print(response_text)
# data = json.loads(response_text)
# parsed_response = AssessmentResultPerCriteria(**data)
# print(f"Result: {parsed_response.result}\n")
# print(f"Explanation: \n {parsed_response.explanation}")

In [ ]:
# Parser
parsed = client.responses.parse(
    model="gpt-4o-mini",
    temperature=0,
    instructions=f"""
    Parse the following response into the provided schema. DO NOT change the content of the response.
    """,
    input=[{"role": "user", "content": [{"type": "input_text", "text": f"Response: {response_text}"}]}],
    text_format=AssessmentResultPerCriteria,
)

print(f"Result: {parsed.output_parsed.result}\n")
print(f"Explanation: \n {parsed.output_parsed.explanation}")